In [ ]:
%pip install -q git+https://github.com/google-research/vision_transformer

In [ ]:
import os
import jax
import numpy as np
import pandas as pd
from vit_jax import models

import requests
from io import BytesIO
from PIL import Image
from pprint import pprint # for neatly formatting print outputs

In [ ]:
# Initialize model (currently available models: LiT-B16B, LiT-B16B_2, LiT-L16L, LiT-L16S, LiT-L16Ti)
model_name = 'LiT-L16Ti'

lit_model = models.get_model(model_name)
tokenizer = lit_model.get_tokenizer()
image_preprocessing = lit_model.get_image_preprocessing()
lit_variables = lit_model.load_variables()

In [ ]:
# Function that returns the top k results from similarity search (using Pinecone's cosine similarity)
def image_embedding(url):
  response = requests.get(url, stream=True)
  image = Image.open(BytesIO(response.content))

  if image.mode == 'RGBA':
      image = image.convert('RGB')

  image = image.resize((500, 500))
  image = np.array(image)
  preprocessed_images = image_preprocessing([image])

  image_features, _, _ = lit_model.apply(lit_variables, images=preprocessed_images)
  # print(image_features.tolist()[0])

  return image_features.tolist()[0]

In [ ]:
def text_embeddings(texts):
    query_tokens = tokenizer(texts)
    _, query_features, _ = lit_model.apply(lit_variables, tokens=query_tokens)
    # for vector in query_features:
    #     print(vector)
    
    return query_features.tolist()